In [13]:
%run ./utils.ipynb

✅ Successfully connected to MongoDB.


In [14]:
client = get_mongo_client(uri)

In [15]:
collection = get_collection(client,"pdf_rag_db","chunks")

In [16]:
# Preview a few documents from the MongoDB collection
for doc in collection.find().limit(5):
    print(f"📄 PDF: {doc.get('pdf_name')}, Page: {doc.get('page_number')}, Chunk: {doc.get('chunk_index')}")
    print(f"📝 Text: {doc.get('chunk_text')[:200]}...\n{'-'*80}")

📄 PDF: metformin2.pdf, Page: 2, Chunk: 0
📝 Text: 2
PRODUCT  MONOGRAPH 
 
 
 
TEVA-METFORMIN 
(metformin hydrochloride) 
 
 
500 mg and 850 mg Tablets 
 
 
 
THERAPEUTIC  CLASSIFICATION 
 
Oral Antihyperglycemic Agent 
 
 
ACTIONS  AND  CLINICAL  PHA...
--------------------------------------------------------------------------------
📄 PDF: metformin2.pdf, Page: 2, Chunk: 1
📝 Text: when there is insulin secretion. Metformin, at therapeutic doses, does not cause hypoglycemia 
when used alone in man or in the non-diabetic animal, except when using a near lethal dose.  
Metformin h...
--------------------------------------------------------------------------------
📄 PDF: metformin2.pdf, Page: 3, Chunk: 0
📝 Text: 3
in urine at high renal clearance rate of about 450 mL/min. The initial elimination of metformin is 
rapid with a half-life varying between 1.7 and 3 hours. The terminal elimination phase 
accounting...
-------------------------------------------------------------------------------

In [17]:
import chromadb
from chromadb.utils import embedding_functions

In [18]:
import chromadb
from chromadb.config import Settings

def store_embedded_chunks_to_chroma(collection):
    """
    Reads chunks from MongoDB collection, embeds them, and stores them in ChromaDB.
    """

    # 1. Init Chroma client (serverless)
    chroma_client = chromadb.Client()

    # 2. Create or get Chroma collection
    chroma_collection = chroma_client.get_or_create_collection(name="pdf_chunks")

    # 3. Prepare data for Chroma
    ids = []
    documents = []
    metadatas = []

    for doc in collection.find():
        # Unique ID per chunk
        chunk_id = f"{doc['pdf_name']}_{doc['page_number']}_{doc['chunk_index']}"

        # Add data
        ids.append(chunk_id)
        documents.append(doc["chunk_text"])
        metadatas.append({
            "pdf_name": doc["pdf_name"],
            "page_number": doc["page_number"],
            "chunk_index": doc["chunk_index"]
        })

    # 4. Add to ChromaDB
    chroma_collection.add(
        ids=ids,
        documents=documents,
        metadatas=metadatas
    )

    print(f"✅ Stored {len(documents)} chunks in ChromaDB.")

In [19]:
# store_embedded_chunks_to_chroma(collection)

In [20]:
# view_all_chunks("pdf_chunks")